In [1]:
from src.functions import *
from scipy.optimize import curve_fit
from joblib import Parallel, delayed

In [2]:
method = 'quantum'

In [3]:
methods = ['random', 'local', 'quantum', 'mismatched']

In [4]:
T = 0.5
num_random_models = 100

In [5]:
qubits_range = np.array([3, 4, 5, 6, 7, 8, 9, 10])
len_qubits_range = len(qubits_range)

In [6]:
delta_arr = np.zeros((len_qubits_range, num_random_models))

for n_index, n in tqdm(enumerate(qubits_range)):
    def compute_delta(seed, method):
        m = RandomIsingModel(n, seed=seed) # set the seed for reproducibility
        if method == 'quantum':
            proposal_mat = get_proposal_mat_quantum_layden(m)
        elif method == 'local':
            proposal_mat = get_proposal_mat_local(m)
        elif method == 'random':
            proposal_mat = get_proposal_mat_random(m)
        elif method == 'mismatched':
            m_mismatched = RandomIsingModel(n, seed=seed+num_random_models) # set the seed for reproducibility
            proposal_mat = get_proposal_mat_quantum_layden(m_mismatched)
        elif method == 'fixed_gamma':
            proposal_mat = get_proposal_mat_quantum_layden(m, gamma_lims=(0.05, 0.05), gamma_steps=1)
        P = get_transition_matrix(m, T, proposal_mat)
        return get_delta(P)

    delta_arr[n_index] = Parallel(n_jobs=-1)(delayed(compute_delta)(seed, method) for seed in range(num_random_models))

0it [00:00, ?it/s]

1it [00:02,  2.24s/it]

2it [00:02,  1.04s/it]

3it [00:02,  1.35it/s]

4it [00:04,  1.03it/s]

5it [00:17,  5.32s/it]

6it [02:43, 53.13s/it]

7it [32:46, 625.43s/it]

In [ ]:
# Fit function
def fit_func(n, a, k):
    return a * 2**(-k * n)

In [ ]:
# Take an average over random Ising instances
delta_arr_avg = np.mean(delta_arr, axis=1)
delta_arr_std = np.std(delta_arr, axis=1)

In [ ]:
popt, pcov = curve_fit(fit_func, qubits_range, delta_arr_avg, maxfev=5000)
fit = fit_func(qubits_range, *popt)
k, k_err = popt[1], np.sqrt(pcov[1, 1])

In [ ]:
qubit_sweep = {'method': method,
               'T': T,
               'qubits_range': qubits_range.tolist(),
               'num_random_models': num_random_models,
               'delta': delta_arr.tolist(),
               'delta_avg': delta_arr_avg.tolist(),
               'delta_std': delta_arr_std.tolist(),
               'popt': popt.tolist(),
               'pcov': pcov.tolist(),
               'fit': fit.tolist(),
               'k': k,
               'k_err': k_err
               }

save_in_json(qubit_sweep, f'data/qubit_sweep_{method}.json')